In [31]:
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
from obspy.core.event import read_events
import os
import pandas as pd
from obspy.core.event.event import EventDescription
from obspy.core.event import Catalog
from Region_distance import find_region

In [2]:
sta = pd.read_csv("stations.csv", error_bad_lines=False,
                   names=["net", "name", "lat", "lon", "z",
                          "cod", "country"], header=None)

def get_loc_cha(station, sta):
    """ Devuelve el codigo de localización, el canal y la red de una
    estación dada. Los datos se toman del dataframe de pandas 
    alimentado por el fichero statios.csv
    
    :type station: str
    :param station: Nombre de la estación
    :type sta: Pandas Dataframe
    :param: Dataframe de pandas con la información de las estaciones 
    que adquiere el SGC. 
    """
    
    loc_cod = 0
    net = ""
    
    columns = ["name", "net", "cod"]
    
    # quitando espacios antes y despues
    for col in columns:
        sta[col] = sta[col].apply(lambda x: x.strip())


    
    df_sta = sta[sta["name"]==station]
    
    # se prueba con los codigos de localizacion posibles priorizando
    # los banda ancha (00)
    codes = ["00", "20", "10", "40"]
    
    cha = {"00":"HHZ", "20":"EHZ", "10":"HNZ", "40":"HLZ", 0:""}
    
    "Si la estacion se encuentra en el data frame:"
    if not df_sta.empty:
        for cod in codes:
            loc_cod = df_sta[sta["cod"] == cod]["cod"].any()
            #print(loc_cod)
            if loc_cod != 0: break

        #print(station)
        net = df_sta["net"].values[0]
    
    return str(loc_cod), cha[loc_cod], net
    
    

In [34]:


def sfiles2xml(xml_name="cat.xml", folder="sfiles/"):
    cat = Catalog()
    
    sfiles = [i for i in os.listdir(folder) if os.path.isfile(os.path.join(folder,i))]
    for sfile in sfiles:
            #if sfile[10:13] == "L.S":
            print(sfile, len(cat))
            event = read_events(folder+sfile, format="NORDIC",
                                encoding='latin-1')
            
            # seleccionando el origen
            o = event.events[0].origins[0]
            
            o.creation_info.agency_id = "SGC"
            o.creation_info.author = "seisan@RSNC.dnl"
            o.creation_info.creation_time = UTCDateTime()

            region = find_region([o.latitude, o.longitude])
            print(region)
            event[0].event_descriptions[0] = EventDescription(text=region,
                                                           type="region name")

            for pick in event.events[0].picks:

                station = pick.waveform_id.station_code

                if station[:4] != "SPEC":
                    # asignando a cada picada el código de localización
                    # el canal y la red
                    loc, cha, net = get_loc_cha(station, sta)

                    pick.waveform_id.location_code = loc
                    pick.waveform_id.channel_code = cha
                    pick.waveform_id.network_code = net

            # agregando el evento al catalogo
            cat.append(event.events[0])
        # escribiendo el catalogo en formato xml SEISCOMP3
    cat.write(xml_name, format="SC3ML")

In [4]:
#cat.plot(projection="local")
#cat.filter("time > 2018-02-01T00:16",
           #"time < 2018-02-01T00:18")
    

In [35]:
# sin paralelizar
from time import time
ti = time()
sfiles2xml("cat_20s_dec_2017.xml")
tf = time()
print("Tiempo de ejecución: ",(tf-ti))

21-1918-32L.S201712 0
PITAL - HUILA


/home/daniel/anaconda2/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 169.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 168.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 150.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 153.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 131.0 in header, currently unsupported
  war

25-0009-48L.S201712 1
LOS SANTOS - SANTANDER
23-1645-57L.S201712 2
LA ARGENTINA - HUILA
27-2115-32L.S201712 3


/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 90.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 134.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 139.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 123.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 119.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)
/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 122.0 in header, currently unsup

SAN ALBERTO - CESAR
25-1536-32L.S201712 4
ZAPATOCA - SANTANDER


...

/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 165.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)


CAICEDONIA - VALLE
25-1822-13L.S201712 171
ZAPATOCA - SANTANDER
24-2053-25L.S201712 172
RIOHACHA - LA_GUAJIRA
20-0409-17L.S201712 173
DABEIBA - ANTIOQUIA
20-0651-28L.S201712 174
EL DOVIO - VALLE
29-0529-04L.S201712 175
BUENAVENTURA - VALLE
24-1622-09L.S201712 176
BETULIA - SANTANDER
21-0202-19L.S201712 177
ZAPATOCA - SANTANDER
22-0336-57L.S201712 178
LOS SANTOS - SANTANDER
23-0605-05L.S201712 179
MURINDO - ANTIOQUIA
23-0616-43L.S201712 180
GUADUAS - CUNDINAMARCA
21-1413-09L.S201712 181
ZAPATOCA - SANTANDER
28-1140-53L.S201712 182
LA JAGUA DE IBIRICO - CESAR
25-2026-18L.S201712 183
LOS SANTOS - SANTANDER
26-2344-46L.S201712 184
MURINDO - ANTIOQUIA
23-0943-06L.S201712 185
FLORIDA - VALLE
26-0340-07L.S201712 186
LOS SANTOS - SANTANDER
26-0533-00L.S201712 187
ZAPATOCA - SANTANDER
24-0014-20L.S201712 188
LLORO - CHOCO
29-0729-38L.S201712 189
DAGUA - VALLE
23-1059-08L.S201712 190
JURADO - CHOCO
24-0430-56L.S201712 191
PUERTO PARRA - SANTANDER
27-0310-40L.S201712 192
LOS SANTOS - SANTANDER
22

/home/daniel/rutinas/obspy/obspy/io/nordic/core.py:756: UserWarning: AIN: 81.0 in header, currently unsupported
  warnings.warn('AIN: %s in header, currently unsupported' % ain)


DAGUA - VALLE
21-1643-04L.S201712 227
BETULIA - SANTANDER
28-0633-03L.S201712 228
LOS SANTOS - SANTANDER
26-1356-06L.S201712 229
DABEIBA - ANTIOQUIA
21-0728-17L.S201712 230
DAGUA - VALLE
23-1202-31L.S201712 231
ZAPATOCA - SANTANDER
28-0604-00L.S201712 232
PIEDECUESTA - SANTANDER
25-0935-30L.S201712 233
EL CARMEN - SANTANDER
27-0643-41L.S201712 234
BETULIA - SANTANDER
29-0346-24L.S201712 235
LOS SANTOS - SANTANDER
27-1709-36L.S201712 236
MOSQUERA - NARINO
22-1042-01L.S201712 237
LOS SANTOS - SANTANDER
26-0342-47L.S201712 238
ZAPATOCA - SANTANDER
25-0522-18L.S201712 239
PUERTO TRIUNFO - ANTIOQUIA
22-0421-42L.S201712 240
JURADO - CHOCO
22-1847-17L.S201712 241
ARGELIA - VALLE
29-0704-46L.S201712 242
LA SIERRA - CAUCA
20-0708-31L.S201712 243
LOS SANTOS - SANTANDER
21-0339-49L.S201712 244
LOS SANTOS - SANTANDER
23-1950-25L.S201712 245
UBATE - CUNDINAMARCA
28-0228-51L.S201712 246
EL DOVIO - VALLE
22-0900-57L.S201712 247
EL CARMEN - SANTANDER
20-1026-14L.S201712 248
ZAPATOCA - SANTANDER
24-214

ALGECIRAS - HUILA
21-0849-01L.S201712 411
LA BELLEZA - SANTANDER
23-1904-28L.S201712 412
BUGA - VALLE
29-0843-52L.S201712 413
SABANA DE TORRES - SANTANDER
28-1407-43L.S201712 414
BETULIA - SANTANDER
25-0615-44L.S201712 415
EL CHARCO - NARINO
26-2015-43L.S201712 416
DABEIBA - ANTIOQUIA
20-0349-03L.S201712 417
LOS SANTOS - SANTANDER
26-1133-49L.S201712 418
ARENAL - BOLIVAR
27-2346-55L.S201712 419
YACOPI - CUNDINAMARCA
23-0706-12L.S201712 420
DABEIBA - ANTIOQUIA
25-0758-37L.S201712 421
EL PLAYON - SANTANDER
24-1220-52L.S201712 422
PALESTINA - HUILA
27-0434-20L.S201712 423
LOS SANTOS - SANTANDER
23-1227-41L.S201712 424
EL PLAYON - SANTANDER
21-0804-00L.S201712 425
ZAPATOCA - SANTANDER
25-2113-15L.S201712 426
LOS SANTOS - SANTANDER
25-1028-21L.S201712 427
AMBALEMA - TOLIMA
24-0548-12L.S201712 428
ABREGO - N.SANTANDER
20-0539-08L.S201712 429
EL PENON - SANTANDER
26-0420-55L.S201712 430
LOS SANTOS - SANTANDER
22-0758-01L.S201712 431
ZAPATOCA - SANTANDER
26-2352-00L.S201712 432
SAN JOSE DEL PA

In [66]:
folder = "sfiles/"
sfile = "31-1227-12L.S201701"
sei_cat = read_events(folder+sfile, format="NORDIC",
                                encoding='latin-1')

In [68]:
ev = sei_cat.events[0]
o=ev.origins[0]

In [76]:
#ev.creation_info #RSNC
#ev.event_descriptions #L
o.quality

OriginQuality(standard_error=1.04, azimuthal_gap=99.0)

## Evento desde Seiscomp3

In [38]:
server_IP = 'http://10.100.100.232'
port_fdsn = "8091"
    
starttime = UTCDateTime("2017 01 31 12 26")    
endtime = UTCDateTime("2017 01 31 12 28")

client = Client(server_IP+":"+port_fdsn)

sc_cat = client.get_events(starttime=starttime, endtime=endtime, includeallmagnitudes=True,
                           includearrivals=True)
len(sc_cat)

1

In [78]:
sc = sc_cat.events[0]
o2=sc.origins[0]
o2.quality

OriginQuality
	   associated_phase_count: 30
	         used_phase_count: 24
	 associated_station_count: 30
	       used_station_count: 30
	           standard_error: 0.4
	            azimuthal_gap: 99.0

In [54]:
a = o2.arrivals[0]

In [27]:
o2=sc.origins[0]
o2.creation_info.agency_id = "SG"
type(o2.longitude)

float

In [ ]:
ls sfiles/28-07*

In [ ]:
t = UTCDateTime()
print(t)